In [1]:
pip install numpy

  Using cached numpy-2.3.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.1 MB 245.8 kB/s eta 0:00:54
   ---------------------------------------- 0.0/13.1 MB 245.8 kB/s eta 0:00:54
   ---------------------------------------- 0.1/13.1 MB 467.6 kB/s eta 0:00:28
   ---------------------------------------- 0.1/13.1 MB 450.6 kB/s eta 0:00:29
    --------------------------------------- 0.2/13.1 MB 615.9 kB/s eta 0:00:21
    --------------------------------------- 0.3/13.1 MB 774.0 kB/s eta 0:00:17
   - -------------------------------------- 0.4/13.1 MB 928.4 kB/s eta 0:00:14
   - -------------------------------------- 0.4/13.1 MB 970.6 kB/s eta 0:00:14
   - -------------------------------------- 0.5/13.1 MB 982.5 kB/s eta 0:00:13
   - ----


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import re 
import random 
from collections import defaultdict, Counter
import numpy as np

In [3]:
def preprocess_text(text):
    global start_tokens
    
    sentences = re.split(r'[.!?]\s*|\n+', text)
    
    tokenized_sentences = []
    for sentence in sentences:
        if not sentence.strip():
            continue
            
        words = re.findall(r'[а-яё]+|[А-ЯЁ][а-яё]*', sentence.lower())
        if len(words) > 1:  # Игнорир слишком коротких предложений
            tokenized_sentences.append(words)
            # Сохранить начальные слова для генерации
            start_tokens.append(words[0])
    
    return tokenized_sentences

In [4]:
def build_vocabulary(tokenized_sentences):
    global unigrams, bigrams, vocab
    
    # Униграммы и словарь
    for sentence in tokenized_sentences:
        for word in sentence:
            unigrams[word] += 1
            vocab.add(word)
    
    # Биграммы
    for sentence in tokenized_sentences:
        for i in range(len(sentence) - 1):
            current_word = sentence[i]
            next_word = sentence[i + 1]
            bigrams[current_word][next_word] += 1

In [5]:
def calculate_probabilities():
#Вычисление вероятностей для биграмм
    global bigram_probs, bigrams
    
    for current_word, next_words in bigrams.items():
        total_count = sum(next_words.values())
        for next_word, count in next_words.items():
            bigram_probs[current_word][next_word] = count / total_count